In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

# Define MLP

In [3]:
class ResNetXIn(nn.Module):
    '''Resnet implementation accepting inputs of varied sizes'''

    def __init__(self, in_channels = 3, cifar = True, size = 50, n_classes = 10, pretrain = False):
    #def __init__(self, size, pretrain, in_channels=1):
        super(ResNetXIn, self).__init__()

        # bring resnet
        if size == 18:
            self.model = models.resnet18(pretrained=pretrain)
        if size == 34:
            self.model = models.resnet34(pretrained=pretrain)
        if size == 50:
            self.model = models.resnet50(pretrained=pretrain)
        if size == 101:
            self.model = models.resnet101(pretrained=pretrain)
        if size == 152:
            self.model = models.resnet152(pretrained=pretrain)

        # your case
        self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        #self.model.fc = nn.Linear(self.model.fc.in_features, n_classes)

    def forward(self, x):
        return self.model(x)

    def set_writer(self, writer):
        """ Set writer, used for bookkeeping."""
        self.writer = writer

In [7]:
net = ResNetXIn(in_channels = 1, cifar=False)

In [12]:
def load_checkpoint(net, file_path):
    #file_path = os.path.join(ckpt_dir, filename)
    if os.path.isfile(file_path):

        checkpoint = torch.load(file_path, map_location=torch.device('cpu'))

        best_valid_loss = checkpoint['valid_loss']
        epoch_counter = checkpoint['epoch']

        net.load_state_dict(checkpoint['model_states']['net'])

        #self.optim.load_state_dict(checkpoint['optim_states']['optim'])
        print("=> loaded checkpoint '{} (epoch {})'".format(file_path, epoch_counter))
    else:
        print("=> no checkpoint found at '{}'".format(file_path))
    return best_valid_loss, epoch_counter

In [21]:
bvl, ec = load_checkpoint(net, 'checkpoints/molar1/best')

=> loaded checkpoint 'checkpoints/molar1/best (epoch 11)'


In [25]:
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torchvision.transforms.functional as F
from torch.nn.functional import interpolate
from torchvision.datasets import CIFAR10
from sklearn.preprocessing import OneHotEncoder
import argparse

import SimpleITK as sitk

In [24]:
class DentalImagingDataset(Dataset):

    def __init__(self, root_dir,
                  csv_file, transform=None, considered_class = 1, ROI_size = 256): # considered_class 1,2 or 3 (see annotations.csv)

        self.annotations = pd.read_csv(csv_file)
        self.annotations = self.annotations[self.annotations['molar_yn'] == 1].iloc[:, [0, 2, 3, 4]]

        self.rootdir = root_dir
        self.transform = transform
        self.considered_class = considered_class
        self.ROI_size = ROI_size

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, item):

        # Image
        file_name = self.annotations.iloc[item, 0] + '.dcm'
        mask_name = file_name[:-4] + '.gipl'
        file_path = os.path.join(self.rootdir, file_name)
        mask_path = os.path.join(self.rootdir, mask_name)
        image = sitk.GetArrayFromImage(sitk.ReadImage(file_path))
        image = image.astype(float).reshape(image.shape[1], image.shape[2])
        mask = sitk.GetArrayFromImage(sitk.ReadImage(mask_path))
        mask = mask.astype(float).reshape(mask.shape[1], mask.shape[2])

        # ############### TRAIN WITH ROIS ##################
        # # Basic transforms: ROI extraction, to tensor, normalize
        # image = utils.extract_ROI(image, mask, self.ROI_size).astype(np.int64)
        # image = torch.Tensor(image)
        # image = image.unsqueeze(0)
        # normalize = utils.Normalize()
        # image = normalize(image)
        # ##################################################

        #################### TRAIN WITH PATCHES #################
        # Basic transforms: ROI extraction, to tensor, normalize
        image = torch.Tensor(image)
        image = image.unsqueeze(0)
        image = Normalize()(image)
        image = Interpolate(window_size=256)(image)
        ##########################################################

        if self.transform is not None:
            image = self.transform(image)

        # Labels
        if self.considered_class == 1:
            pad = 1
        else:
            pad = 4
        labels = int(self.annotations.iloc[item, self.considered_class]-pad)

        sample = [image, labels]

        return sample

def returnOPGdata(args):

    ## remember to add RandomResizedCrop to the server (solve the problem by running)

    dset_dir = args.dset_dir  ## directory that contains images
    csv_file = args.csv_file  ## csv file with annotations: File_name/ molar_yn/ class_1/ class_2/ class_3
    considered_class = args.considered_class
    ROI_size = args.ROI_size

    batch_size = args.batch_size
    test_portion = args.test_portion
    num_workers = args.num_workers

    def split_sets(dset, test_portion):
        # Creating data indices for training and validation splits:
        dataset_size = len(dset)
        test_size = int(dataset_size * test_portion)
        train_size = int(dataset_size - test_size)

        valid_set, test_set = random_split(dset, [train_size, test_size])

        return valid_set, test_set

    #dset = DentalImagingDataset(root_dir=dset_dir, csv_file=csv_file,
    #                            transform=None, considered_class=considered_class, ROI_size=ROI_size)

    #train_set, test_set = split_sets(dset, test_portion)

    ####################################################
    ####################################################
    train_set = DentalImagingDataset(root_dir=dset_dir, csv_file=csv_file,
                                transform=None, considered_class=considered_class, ROI_size=ROI_size)
    test_csv = csv_file[:-21]+'test_annotations.csv'
    test_set = DentalImagingDataset(root_dir=dset_dir, csv_file=test_csv,
                                transform=None, considered_class=considered_class, ROI_size=ROI_size)
    ####################################################
    ####################################################

    train_loader = DataLoader(train_set,
                              batch_size=batch_size,
                              shuffle=False,
                              num_workers=num_workers,
                              pin_memory=True,
                              drop_last=False)

    test_loader = DataLoader(test_set,
                             batch_size=batch_size,
                             shuffle=False,
                             num_workers=num_workers,
                             pin_memory=True,
                             drop_last=False)

    return train_loader, test_loader

class Normalize(object):
    """Normalizes the image tensor"""

    def __call__(self, image):
        mean = 12526.53
        std = 30368.829025877254
        image = F.normalize(image, [mean], [std])

        return image

class Interpolate(object):
    """Reduces size of image"""

    def __init__(self, window_size):
        self.window_size = window_size

    def __call__(self, image):
        image = interpolate(image.unsqueeze(0), size=(self.window_size, self.window_size))
        image = image.squeeze(0)

        return image

In [26]:
dset_args = argparse.Namespace()
dset_args.dset_dir = '../../molar_detection/data/annotated/patches/images_nmasks_original'
dset_args.csv_file = '../../molar_detection/data/annotated/annotations/all/cv1/train_annotations.csv'
dset_args.considered_class = 1
dset_args.ROI_size = 256

dset_args.batch_size = 64
dset_args.test_portion = 0.2
dset_args.num_workers = 1

In [43]:

train_set = DentalImagingDataset(root_dir=dset_args.dset_dir, csv_file=dset_args.csv_file,
                                transform=None, considered_class=1, ROI_size=256)
test_csv = dset_args.csv_file[:-21]+'test_annotations.csv'
test_set = DentalImagingDataset(root_dir=dset_args.dset_dir, csv_file=test_csv,
                                transform=None, considered_class=1, ROI_size=256)

In [52]:
test_loader = DataLoader(test_set,
                             batch_size=32,
                             shuffle=False,
                             num_workers=1,
                             pin_memory=True,
                             drop_last=False)

In [47]:
tr_loader, val_loader = returnOPGdata(dset_args)

In [48]:
#test_set[0]

In [54]:
test_set

In [56]:
from torch.autograd import Variable

In [59]:
outs = []
for i, (x,l) in enumerate(test_set):
    x = x.unsqueeze(0)
    x = Variable(x)

    out = net(x.float())
    outs.append(out)

In [61]:
len(outs)

34

In [71]:
out_ints = []
for o in outs:
    _, i = o.max(1)
    out_ints.append(i)

In [68]:
_, i = outs[0].max(1)

In [72]:
    out_ints.append(i)

In [6]:
for param in tester.features.parameters():
    param.requires_grad = False

In [14]:
for param in tester.named_parameters():
    #print(param[1].requires_grad)
    if param[1].requires_grad:
        print(param[0])

model.fc.weight
model.fc.bias
l1.weight
l1.bias
l2.weight
l2.bias


In [11]:
tester

ResNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2

In [11]:
list(tester.l1.parameters()) + list(tester.l2.parameters())

[Parameter containing:
 tensor([[ 0.0074, -0.0209, -0.0107,  ..., -0.0047,  0.0159,  0.0169],
         [ 0.0136, -0.0110,  0.0104,  ...,  0.0089,  0.0054,  0.0023],
         [ 0.0221, -0.0040,  0.0039,  ...,  0.0129,  0.0191, -0.0019],
         ...,
         [-0.0146,  0.0108, -0.0167,  ..., -0.0109,  0.0129, -0.0022],
         [-0.0213, -0.0084, -0.0210,  ...,  0.0187,  0.0134,  0.0067],
         [-0.0107, -0.0077,  0.0094,  ..., -0.0170, -0.0120,  0.0131]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0190, -0.0009, -0.0207,  ..., -0.0143, -0.0176,  0.0146],
        requires_grad=True),
 Parameter containing:
 tensor([[ 1.6655e-02,  1.5310e-02,  1.3817e-02,  ..., -1.5510e-02,
           1.6460e-02, -7.7611e-03],
         [-1.0474e-02,  2.4521e-02,  5.7149e-03,  ...,  2.1874e-02,
           3.3414e-03,  3.7410e-03],
         [ 7.9533e-03,  3.0618e-02, -2.2545e-02,  ..., -1.2592e-02,
           2.6652e-02, -3.0690e-02],
         ...,
         [-1.3885e-02, -5.1784e-0

In [16]:
molars = pd.read_csv('../../molar_detection/data/annotated/annotations/all/cv1/train_annotations.csv')

In [21]:
class_1 = np.array(molars[molars['molar_yn'] == 1]['class_1'])
class_2 = np.array(molars[molars['molar_yn'] == 1]['class_2'])

In [23]:
print(len(np.where(class_1 == 1)[0]) / len(class_1))
print(len(np.where(class_1 == 2)[0]) / len(class_1))
print(len(np.where(class_1 == 3)[0]) / len(class_1))

0.4897959183673469
0.272108843537415
0.23809523809523808


In [24]:
print(len(np.where(class_2 == 4)[0]) / len(class_2))
print(len(np.where(class_2 == 5)[0]) / len(class_2))
print(len(np.where(class_2 == 6)[0]) / len(class_2))

0.1360544217687075
0.6462585034013606
0.21768707482993196


In [27]:
from sklearn.metrics import balanced_accuracy_score, accuracy_score

In [34]:
accuracy_score(class_1.astype(int), np.ones(len(class_1), dtype=int))

0.4897959183673469

In [35]:
balanced_accuracy_score(class_1.astype(int), np.ones(len(class_1), dtype=int))

0.3333333333333333

In [31]:
class_1.astype(int)

array([1, 3, 1, 1, 2, 2, 3, 1, 1, 1, 3, 2, 3, 2, 1, 2, 2, 2, 3, 3, 3, 3,
       1, 1, 1, 2, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 3, 2, 2, 1,
       1, 2, 2, 1, 1, 2, 2, 1, 3, 2, 3, 2, 1, 1, 2, 3, 1, 2, 1, 1, 2, 2,
       1, 2, 1, 1, 2, 2, 2, 3, 1, 3, 3, 1, 1, 2, 2, 2, 3, 1, 2, 1, 2, 2,
       1, 1, 1, 1, 2, 3, 2, 1, 1, 1, 3, 3, 1, 2, 1, 2, 1, 1, 3, 1, 3, 1,
       2, 3, 1, 3, 1, 1, 1, 1, 2, 1, 1, 3, 3, 3, 1, 1, 1, 3, 2, 1, 1, 3,
       1, 3, 1, 1, 1, 1, 2, 1, 1, 2, 3, 3, 1, 1, 3])

In [33]:
np.ones(len(class_1), dtype=int)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [36]:
'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet_(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet_, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    
def ResNet50():
    return ResNet_(Bottleneck, [3, 4, 6, 3])

In [37]:
model = ResNet50()

In [38]:
model

ResNet_(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Bottleneck(
      

In [39]:
tester

ResNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2